### Install & Import Packages

In [2]:
%%capture
!pip install "flair" -q

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import flair
from flair.data import Sentence
from flair.datasets import ColumnCorpus
from flair.embeddings import (
    WordEmbeddings, TransformerWordEmbeddings, StackedEmbeddings
)
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
flair.__version__

'0.12.2'

### Import Data

In [4]:
DATA_PATH = "/content/drive/Shareddrives/CIS522-Project/data"
MODEL_PATH = "/content/drive/Shareddrives/CIS522-Project/models"

In [5]:
# Import data into flair using ColumnCorpus
corpus = ColumnCorpus(DATA_PATH, {0: "text", 1: "ner"}, train_file="flair_ner_train_augmented.txt", test_file="flair_ner_test.txt")
# Needed for model initialization
tag_dictionary = corpus.make_label_dictionary(label_type="ner", add_unk=False)
print(tag_dictionary.get_items())

2023-04-15 06:02:07,568 Reading data from /content/drive/Shareddrives/CIS522-Project/data
2023-04-15 06:02:07,572 Train: /content/drive/Shareddrives/CIS522-Project/data/flair_ner_train_augmented.txt
2023-04-15 06:02:07,576 Dev: None
2023-04-15 06:02:07,577 Test: /content/drive/Shareddrives/CIS522-Project/data/flair_ner_test.txt
2023-04-15 06:02:34,037 Computing label dictionary. Progress:


2776it [00:00, 24020.36it/s]

2023-04-15 06:02:34,197 Dictionary created for label 'ner' with 9 values: Drug (seen 5597 times), ADE (seen 4556 times), Reason (seen 1281 times), Strength (seen 964 times), Route (seen 816 times), Frequency (seen 713 times), Form (seen 620 times), Dosage (seen 595 times), Duration (seen 127 times)
['Drug', 'ADE', 'Reason', 'Strength', 'Route', 'Frequency', 'Form', 'Dosage', 'Duration']


### Initialize Weight Dictionary
This is the weight dictionary used by the loss function. The weight for a given entity is set to the ratio between the frequency of the most represented entity and the frequency of the given entity.

In [ ]:
weight_dict = {
    'Drug': 5597/5597,
    'Strength': 5597/964,
    'Form': 5597/620,
    'Frequency': 5597/713,
    'Route': 5597/816,
    'Dosage': 5597/595,
    'Reason': 5597/1281,
    'Duration': 5597/127,
    'ADE': 5597/4556
}

In [7]:
tf_tagger = SequenceTagger.load(f"{MODEL_PATH}/taggers/clinicalbert-crf/final-model.pt")
tf_tagger.weight_dict = weight_dict

2023-04-15 06:02:59,045 SequenceTagger predicts: Dictionary with 39 tags: O, S-Drug, B-Drug, E-Drug, I-Drug, S-Strength, B-Strength, E-Strength, I-Strength, S-Form, B-Form, E-Form, I-Form, S-Frequency, B-Frequency, E-Frequency, I-Frequency, S-Route, B-Route, E-Route, I-Route, S-Dosage, B-Dosage, E-Dosage, I-Dosage, S-Reason, B-Reason, E-Reason, I-Reason, S-Duration, B-Duration, E-Duration, I-Duration, S-ADE, B-ADE, E-ADE, I-ADE, <START>, <STOP>


### Fine-Tune & Evaluate Model

In [8]:
# Initialize trainer
trainer = ModelTrainer(tf_tagger, corpus)

# Train on corpus
trainer.train(
    base_path=f"{MODEL_PATH}/taggers/clinicalbert-crf-augmented",
    train_with_dev=False,
    max_epochs=10,
    learning_rate=0.005,
    mini_batch_size=16,
    embeddings_storage_mode='none'
)

/usr/local/lib/python3.9/dist-packages/flair/trainers/trainer.py:69: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(


2023-04-15 06:03:00,934 ----------------------------------------------------------------------------------------------------
2023-04-15 06:03:00,939 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): TransformerWordEmbeddings(
      (model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(28997, 768)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_fea

100%|██████████| 20/20 [00:13<00:00,  1.52it/s]


2023-04-15 06:06:59,966 Evaluating as a multi-label problem: False
2023-04-15 06:06:59,997 DEV : loss 0.1432962864637375 - f1-score (micro avg)  0.8595
2023-04-15 06:07:00,008 BAD EPOCHS (no improvement): 0
2023-04-15 06:07:00,012 saving best model
2023-04-15 06:07:01,518 ----------------------------------------------------------------------------------------------------
2023-04-15 06:07:25,119 epoch 2 - iter 17/174 - loss 0.24036553 - time (sec): 23.60 - samples/sec: 471.85 - lr: 0.005000
2023-04-15 06:07:46,485 epoch 2 - iter 34/174 - loss 0.23766284 - time (sec): 44.96 - samples/sec: 482.67 - lr: 0.005000
2023-04-15 06:08:10,454 epoch 2 - iter 51/174 - loss 0.23116425 - time (sec): 68.93 - samples/sec: 478.98 - lr: 0.005000
2023-04-15 06:08:31,643 epoch 2 - iter 68/174 - loss 0.23109893 - time (sec): 90.12 - samples/sec: 486.83 - lr: 0.005000
2023-04-15 06:08:53,295 epoch 2 - iter 85/174 - loss 0.23258418 - time (sec): 111.77 - samples/sec: 488.28 - lr: 0.005000
2023-04-15 06:09:17,

100%|██████████| 20/20 [00:11<00:00,  1.77it/s]


...

100%|██████████| 20/20 [00:10<00:00,  1.95it/s]


2023-04-15 06:43:15,013 Evaluating as a multi-label problem: False
2023-04-15 06:43:15,039 DEV : loss 0.049334339797496796 - f1-score (micro avg)  0.9528
2023-04-15 06:43:15,050 BAD EPOCHS (no improvement): 1
2023-04-15 06:43:20,496 ----------------------------------------------------------------------------------------------------
2023-04-15 06:43:24,078 SequenceTagger predicts: Dictionary with 39 tags: O, S-Drug, B-Drug, E-Drug, I-Drug, S-Strength, B-Strength, E-Strength, I-Strength, S-Form, B-Form, E-Form, I-Form, S-Frequency, B-Frequency, E-Frequency, I-Frequency, S-Route, B-Route, E-Route, I-Route, S-Dosage, B-Dosage, E-Dosage, I-Dosage, S-Reason, B-Reason, E-Reason, I-Reason, S-Duration, B-Duration, E-Duration, I-Duration, S-ADE, B-ADE, E-ADE, I-ADE, <START>, <STOP>


100%|██████████| 1588/1588 [13:16<00:00,  1.99it/s]


2023-04-15 06:56:42,619 Evaluating as a multi-label problem: False
2023-04-15 06:56:44,785 0.8869	0.9188	0.9026	0.8296
2023-04-15 06:56:44,786 
Results:
- F-score (micro) 0.9026
- F-score (macro) 0.8172
- Accuracy 0.8296

By class:
              precision    recall  f1-score   support

        Drug     0.8731    0.9410    0.9058     61167
    Strength     0.9357    0.9562    0.9458     42957
        Form     0.9178    0.9228    0.9203     41417
   Frequency     0.8551    0.8580    0.8566     36495
       Route     0.9509    0.9424    0.9466     30583
      Dosage     0.9244    0.9455    0.9348     23506
      Reason     0.7222    0.7490    0.7353      9533
         ADE     0.2378    0.6451    0.3475      1299
    Duration     0.7466    0.7790    0.7625      1982

   micro avg     0.8869    0.9188    0.9026    248939
   macro avg     0.7960    0.8599    0.8172    248939
weighted avg     0.8930    0.9188    0.9051    248939

2023-04-15 06:56:44,791 ---------------------------------------

{'test_score': 0.9025737143759063,
 'dev_score_history': [0.8595224875069406,
  0.8897593732512591,
  0.8976290097629009,
  0.9150253235790659,
  0.9111479028697572,
  0.9299435028248587,
  0.945174186179326,
  0.9435140505251206,
  0.953388618816128,
  0.9527872582480091],
 'train_loss_history': [0.2693294886703613,
  0.22516719765072943,
  0.19442447590181217,
  0.17497128857277183,
  0.1607360814800832,
  0.14557645680775588,
  0.13896159373575337,
  0.1322365663109756,
  0.12198050303097219,
  0.11407735211989971],
 'dev_loss_history': [0.1432962864637375,
  0.11918845027685165,
  0.10443847626447678,
  0.08795062452554703,
  0.09466564655303955,
  0.07021188735961914,
  0.057655636221170425,
  0.05484052002429962,
  0.04831854626536369,
  0.049334339797496796]}

In [ ]:
# Create example sentence
sentence = Sentence("Patients on 40 mg of Topelfate and Topoxy twice a day for stomachache generally suffer from headache")

# Token level predictions
tf_tagger.predict(sentence, force_token_predictions=True)
print(sentence.to_tagged_string())

# Predict tags and print
tf_tagger.predict(sentence)
print(sentence.to_tagged_string())

Sentence[17]: "Patients on 40 mg of Topelfate and Topoxy twice a day for stomachache generally suffer from headache" → ["40"/B-Strength, "mg"/E-Strength, "Topelfate"/S-Drug, "Topoxy"/S-Drug, "twice"/B-Frequency, "a"/I-Frequency, "day"/E-Frequency, "stomachache"/S-Reason, "headache"/S-ADE]
Sentence[17]: "Patients on 40 mg of Topelfate and Topoxy twice a day for stomachache generally suffer from headache" → ["40 mg"/Strength, "Topelfate"/Drug, "Topoxy"/Drug, "twice a day"/Frequency, "stomachache"/Reason, "headache"/ADE]
